In [1]:
from iteru import *

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(SearchControl(found_style={'fillColor': '#3f0', 'color': '#0f0'}, marker=Marker…

In [60]:
aoi = GERD_aoi_dam

In [61]:
# Map.add_layer_widgets(ee.FeatureCollection(aoi),{'color':'blue'},'AOI')

In [91]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filter(ee.Filter.eq('system:time_start',1651289124000))\
    .select(['VV','VH'])\
    .median()

In [92]:
VV_smooth = SAR.select('VV').focal_median(50,'circle','meters').rename('VV_Filtered')

In [93]:
#VV_Filtered = ee.Image(toDB(RefinedLee(toNatural(SAR.select(['VV']))))).rename('VV_Filtered')

In [94]:
histogram = VV_smooth.select('VV_Filtered').reduceRegion(               
            reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
            geometry = aoi, 
            scale = 10,
            bestEffort = True   )

In [95]:
otsu_threshold = otsu(histogram.get('VV_Filtered_histogram'))

In [96]:
threshold = otsu_threshold.getInfo()

In [97]:
x = threshold -1.5

In [98]:
threshold

-16.12053153482008

In [99]:
x 

-17.62053153482008

In [100]:
water_mask = VV_smooth.select('VV_Filtered').lt(x).selfMask().rename('water_mask')

In [101]:
water_mask = water_mask.select('water_mask').clip(aoi)

In [102]:
# Map.add_ee_layer(water_mask,{'palette': ['00FFFF', '0000FF'], 'bands':['water_mask']},'SARF')

In [103]:
feature = ee.Image(1).updateMask(water_mask).reduceToVectors(
           geometry = water_mask.geometry(),
           crs ='EPSG:32636',
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
          )

In [104]:
def calc_area(feature):
    
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [105]:
feature = feature.map(calc_area)

In [106]:
lake_feature = feature.sort('Area', False).first()

In [107]:
# Map.add_layer_widgets(ee.FeatureCollection(lake_feature.geometry()),{'color':'blue'},'Reservoir')

In [108]:
lake_area = lake_feature.geometry().area(maxError = 1) 

In [109]:
lake_area.getInfo()/1e6

257.16250961384105

In [110]:
lake_dem = elevation_dataset.clip(lake_feature)

In [111]:
max_ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 30, 
                         crs = 'EPSG:32636',
                        maxPixels = 1e11).get('DSM')

In [112]:
water_level = max_ele.getInfo()

In [113]:
water_level

587

In [114]:
elevations = lake_dem.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=lake_dem.geometry(),
        maxPixels=1e11,
        scale=30,
        crs = 'EPSG:32636',
        bestEffort = True
    ).get('DSM')

In [115]:
elev_pixles_num = ee.List(elevations).length()

In [116]:
elev_sum = ee.List(elevations).reduce(ee.Reducer.sum())

In [117]:
((water_level*elev_pixles_num.getInfo()-elev_sum.getInfo())*900)/(1e9)

9.3151269

In [118]:
test = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filterDate('2022-01-01','2022-05-05')\
    .select(['VV','VH'])\
    

In [119]:
get_imgCol_dates_dict(test)

{'2022-01-12': 1641957924000,
 '2022-01-24': 1642994724000,
 '2022-02-05': 1644031523000,
 '2022-03-01': 1646105123000,
 '2022-03-13': 1647141923000,
 '2022-03-25': 1648178723000,
 '2022-04-06': 1649215523000,
 '2022-04-18': 1650252324000,
 '2022-04-30': 1651289124000}